## Exploratory Data Analysis
>This notebook shows some basic Exploratory Data Analysis about the dataset

### Objectives
0. Clean data
1. Which book is the most popular?
2. Which author is the most popular?
3. Which number wrote the biggest number of books?
4. Is number of pages correlated with ratings or number of reviews?
5. Which years had the biggest number of books written?
6. Is there tendency to reduce number of pages in nowaday books?

### Import libraries

In [1]:
import pandas as pd 
import polars as pl 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline
sns.set_style('whitegrid')

### Load the data

In [2]:
books_df = pl.DataFrame(schema=pl.read_csv('dataset/book1000k-1100k.csv').schema)
books_df.head()

Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews
i64,str,str,str,f64,i64,i64,i64,str,str,str,str,str,str,str,i64,str,i64,str,i64


In [17]:
books_df.replace('PublishDay', books_df['PublishDay'].shrink_dtype())
books_df.replace('PublishMonth', books_df['PublishMonth'].shrink_dtype())
books_df.replace('PublishYear', books_df['PublishYear'].shrink_dtype())
books_df.schema

{'Id': Int64,
 'Name': Utf8,
 'Authors': Utf8,
 'ISBN': Utf8,
 'Rating': Float64,
 'PublishYear': Int32,
 'PublishMonth': Int8,
 'PublishDay': Int8,
 'Publisher': Utf8,
 'RatingDist5': Utf8,
 'RatingDist4': Utf8,
 'RatingDist3': Utf8,
 'RatingDist2': Utf8,
 'RatingDist1': Utf8,
 'RatingDistTotal': Utf8,
 'CountsOfReview': Int64,
 'Language': Utf8,
 'pagesNumber': Int64,
 'Description': Utf8,
 'Count of text reviews': Int64,
 'PagesNumber': Int64}

In [18]:
books_df[['PublishDay', 'PublishMonth', 'PublishYear']].describe()

describe,PublishDay,PublishMonth,PublishYear
str,f64,f64,f64
"""count""",1.850198e6,1.850198e6,1.850198e6
"""null_count""",0.0,0.0,0.0
"""mean""",7.987328,7.700395,1997.841519
"""std""",8.431971,7.756715,87.897239
"""min""",1.0,1.0,1.0
"""max""",31.0,31.0,65535.0
"""median""",5.0,6.0,2000.0


- Min and max year looks strange

- Max month is 31, also, mean of day seems strange

In [19]:
books_df['PublishYear'].unique()

PublishYear
i32
1
8
162
199
200
202
208
299
1192


- Year 1, 8, 162, 199, 200, 202, 208, 299, 2030, 2035, ... are wrong

- Year 2022 and 2021 need further inspecting

In [20]:
books_df['PublishYear'].value_counts()

PublishYear,counts
i32,u32
1376,1
1952,145
1896,2
1920,145
20040,1
1880,1
1912,17
1192,1
2016,1069


Let's just take the books from 1700 to 2021 because others seem suspicious

In [21]:
books_df = books_df.sort('PublishYear')

In [22]:
low = books_df.filter(books_df['PublishYear'] < 1700).shape[0]
high = books_df.filter(books_df['PublishYear'] > 2021).shape[0]

In [23]:
books_df = books_df[low:len(books_df)-high]

In [24]:
books_df['PublishYear'].value_counts()

PublishYear,counts
i32,u32
1730,1
1753,2
1824,1
1825,1
1833,1
1835,2
1836,1
1837,1
1838,1


#### Publisher

In [25]:
books_df.filter(books_df['Publisher'].is_null()).head()

Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews,PagesNumber
i64,str,str,str,f64,i32,i8,i8,str,str,str,str,str,str,str,i64,str,i64,str,i64,i64
3088068,"""Godey's Lady's…","""Various""","""1426484372""",2.0,1851,10,28,null,"""5:0""","""4:0""","""3:0""","""2:2""","""1:0""","""total:2""",0,null,null,"""This is a pre-…",null,220
1424633,"""An Appeal in v…","""Alfred H. Love…","""1429753579""",0.0,1862,1,1,null,"""5:0""","""4:0""","""3:0""","""2:0""","""1:0""","""total:0""",0,null,24,null,0,null
2619182,"""French for Mas…","""Jean-Paul Vale…","""0669200832""",3.4,1900,1,1,null,"""5:0""","""4:4""","""3:0""","""2:0""","""1:1""","""total:5""",1,"""eng""",null,null,null,480
2802465,"""Foundations of…","""McDougal Litte…","""0669403636""",0.0,1900,1,1,null,"""5:0""","""4:0""","""3:0""","""2:0""","""1:0""","""total:0""",0,null,null,null,null,544
194155,"""Dime Uno Cuade…","""Fabián A. Sama…","""0669433470""",4.0,1900,1,1,null,"""5:4""","""4:0""","""3:2""","""2:1""","""1:0""","""total:7""",0,null,0,null,null,null


In [26]:
books_df['Publisher'].n_unique()

79419

A lot of books. Also books with good ratings, I should not remove them

In [27]:
# Which publisher issued the biggest variety of books
books_df['Publisher'].value_counts()

Publisher,counts
str,u32
"""Vintage Press""",1
"""Carraig Books""",1
"""Grove's Dictio…",4
"""Éditions La Dé…",3
"""Chrysalis Chil…",16
"""Roberta Gregor…",1
"""Quai Voltaire""",13
"""World Almanac …",44
"""Blake Publishi…",2


- Missing value: Not Avail, Unknown, Not Specified, Not Applicable, ...

#### RatingDist (1, 2, 3, 4, 5, total)

In [28]:
books_df.head(3)

Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews,PagesNumber
i64,str,str,str,f64,i32,i8,i8,str,str,str,str,str,str,str,i64,str,i64,str,i64,i64
3098402,"""Three Distinct…","""Samuel Pritcha…","""1419163108""",0.0,1730,12,1,"""Kessinger Publ…","""5:0""","""4:0""","""3:0""","""2:0""","""1:0""","""total:0""",0,null,null,null,null,80
2448024,"""Discovering Ge…","""Michael Serra""","""1559532009""",3.67,1753,1,1,"""Kendall/Hunt P…","""5:1""","""4:1""","""3:0""","""2:1""","""1:0""","""total:3""",1,null,null,null,null,834
4265642,"""Self Esteem In…","""Lila Swell""","""0840360134""",0.0,1753,1,1,"""Kendall/Hunt P…","""5:0""","""4:0""","""3:0""","""2:0""","""1:0""","""total:0""",0,null,null,null,null,170


Get rid of redundant parts like '5:', '4:', 'total:', ...

In [29]:
books_df = books_df.with_columns(books_df['RatingDist1'].apply(lambda rating: rating.split(':')[1]).cast(pl.Int32))
books_df = books_df.with_columns(books_df['RatingDist2'].apply(lambda rating: rating.split(':')[1]).cast(pl.Int32))
books_df = books_df.with_columns(books_df['RatingDist3'].apply(lambda rating: rating.split(':')[1]).cast(pl.Int32))
books_df = books_df.with_columns(books_df['RatingDist4'].apply(lambda rating: rating.split(':')[1]).cast(pl.Int32))
books_df = books_df.with_columns(books_df['RatingDist5'].apply(lambda rating: rating.split(':')[1]).cast(pl.Int32))
books_df = books_df.with_columns(books_df['RatingDistTotal'].apply(lambda rating: rating.split(':')[1]).cast(pl.Int32))
books_df[['RatingDistTotal', 'RatingDist1', 'RatingDist2', 'RatingDist3', 'RatingDist4', 'RatingDist5']].describe()

describe,RatingDistTotal,RatingDist1,RatingDist2,RatingDist3,RatingDist4,RatingDist5
str,f64,f64,f64,f64,f64,f64
"""count""",1.850149e6,1.850149e6,1.850149e6,1.850149e6,1.850149e6,1.850149e6
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",4079.614346,94.402791,207.246389,754.638235,1305.846063,1717.480868
"""std""",71662.082866,2227.034126,3567.509688,11238.758305,20767.199856,37288.463405
"""min""",-2.0,0.0,0.0,0.0,0.0,-2.0
"""max""",7.094687e6,550388.0,544093.0,1.013165e6,1.912159e6,4.608992e6
"""median""",5.0,0.0,0.0,1.0,2.0,1.0


Looks better

#### Counts of reviews and Count of text reviews

In [30]:
books_df['CountsOfReview'].describe()

statistic,value
str,f64
"""min""",0.0
"""max""",154447.0
"""null_count""",0.0
"""mean""",11.580386
"""std""",295.280151
"""count""",1.850149e6


In [31]:
books_df['CountsOfReview'].value_counts()

CountsOfReview,counts
i64,u32
1552,3
40,1185
240,48
2504,1
640,9
2632,1
1024,4
2744,1
2088,1


In [32]:
books_df = books_df.rename({'Count of text reviews': 'CountOfTextReviews'})
books_df['CountOfTextReviews'].describe()

statistic,value
str,f64
"""min""",0.0
"""max""",43652.0
"""null_count""",1.440462e6
"""mean""",7.172849
"""std""",124.840352
"""count""",1.850149e6
